In [15]:
# Importing the necessary libraries for AI2-THOR to run

!pip install --upgrade ai2thor ai2thor-colab &> /dev/null
import ai2thor
import ai2thor_colab
import pickle
import os.path
import time
from typing import Dict, List

from ai2thor.controller import Controller
from ai2thor_colab import (
    plot_frames,
    show_objects_table,
    side_by_side,
    overlay,
    show_video
)

# Class for controlling robot navigation. This is where we will have all the navigation commands.
# This has NOT yet got the LLM connected, but merely a set of tools to move the robot and to interact
# with the simulation environment.
class RobotNavigationControl:
    is_DEBUG = False

    # Starts server
    def start_ai2_thor(self):
        ai2thor_colab.start_xserver()
        "AI2-THOR Version: " + ai2thor.__version__

    # Initialises controller
    def initialise_controller(self):
        self.controller = Controller(
            agentMode="default",
            visibilityDistance=1.5,
            scene="FloorPlan209",
            # image modalities
            #renderDepthImage=False,
            #renderInstanceSegmentation=False,
            # camera properties
            width=400,
            height=400,
            fieldOfView=120,
            # step sizes
            gridSize=0.25,
            snapToGrid=True,
            #rotateStepDegrees=15,
        )

        # If debug is enabled, then print scene name and a few other things.
        if (self.is_DEBUG):
            event = self.controller.step(action="RotateRight")
            metadata = event.metadata
            print(event, event.metadata.keys())
            print("sceneName : " + self.controller.last_event.metadata["sceneName"])
            #print("actionReturn : " + controller.last_event.metadata["actionReturn"])

    # Print a table of all objects in the scene
    def show_all_objects(self):
        show_objects_table(self.controller.last_event.metadata['objects'])
        print(self.controller.last_event.metadata['objects'])
    
    # Rotate left by given number of degrees degrees
    def rotate_left(self, deg):
        #event = controller.step("MoveBack")
        #plot_frames(controller.last_event)
        frames = []
        for _ in range(deg // 5):
            frames.append(self.controller.step(action="RotateLeft", degrees=5).frame)
            time.sleep(0.05)        
        
    # Store visible objects in the self.visible_objects collection and print them out if needed
    def get_visible_objects(self, print_objects = False):
        objects = self.controller.last_event.metadata['objects']
        visible_objects = []

        for obj in objects:
            if obj['visible']:
                if print_objects:
                    print(obj['objectType'] + " : " + str(obj['position']))
                visible_objects.append(obj)
                
        return visible_objects
        
    # Navigate to object defined by the name in the input
    def navigate_to_object(self, obj_name):
        visible_objs = self.get_visible_objects()
        
        obj_names = sorted([obj["objectType"] for obj in visible_objs])
        assert obj_name in obj_names
        
        navigate_to = next(obj for obj in visible_objs if obj["objectType"] == obj_name)
        
        #print(navigate_to)
        
        reachable_positions = self.controller.step(action="GetReachablePositions").metadata["actionReturn"]

In [14]:
rnc = RobotNavigationControl()
rnc.start_ai2_thor()
rnc.initialise_controller()
rnc.rotate_left(180)
rnc.get_visible_objects(True)
rnc.navigate_to_object("Vase")


ArmChair : {'x': -4.334003448486328, 'y': 0.001995176076889038, 'z': -3.5289852619171143}
CoffeeTable : {'x': -2.4740657806396484, 'y': 0.0015519559383392334, 'z': -2.4894556999206543}
Floor : {'x': -4.4715800285339355, 'y': -0.055619120597839355, 'z': -2.5462260246276855}
Laptop : {'x': -2.57079815864563, 'y': 0.5791798233985901, 'z': -2.5124337673187256}
RemoteControl : {'x': -4.292991638183594, 'y': 0.33039507269859314, 'z': -2.663956880569458}
Shelf : {'x': -2.475966215133667, 'y': 0.12455198913812637, 'z': -2.486457347869873}
Sofa : {'x': -4.3549981117248535, 'y': 0.00991678237915039, 'z': -2.089966058731079}
Vase : {'x': -2.6081905364990234, 'y': 0.579205334186554, 'z': -2.9408369064331055}
{'name': 'Vase_99a3f189', 'position': {'x': -2.6081905364990234, 'y': 0.579205334186554, 'z': -2.9408369064331055}, 'rotation': {'x': 359.9898986816406, 'y': 359.43280029296875, 'z': -0.0017173961969092488}, 'visible': True, 'isInteractable': True, 'receptacle': False, 'toggleable': False, 'is